# Unit 2 Assignment: Mixture of Experts (MoE) Router + Tool Use

This notebook implements:
- Router (Intent Classifier)
- Technical, Billing, General Experts
- BONUS: Tool Use Expert (Bitcoin Price)


### Section 1: Set up
Includes installation adn loading the api key

In [22]:
!pip install -q groq python-dotenv

In [23]:
import os
from dotenv import load_dotenv
from groq import Groq

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

BASE_MODEL = "llama-3.1-8b-instant"

### Section 2: Define Experts

In [24]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": "You are a Senior Technical Support Engineer. Provide precise and code-focused solutions.",
        "temperature": 0.7
    },
    "billing": {
        "system_prompt": "You are a Billing Support Specialist. Be empathetic and policy-driven.",
        "temperature": 0.7
    },
    "general": {
        "system_prompt": "You are a friendly assistant helping customers.",
        "temperature": 0.7
    }
}

### Section 3: Router Function

In [25]:
def route_prompt(user_input):
    routing_prompt = f"""
Classify this text into one of these categories:
[technical, billing, general, crypto]

Return ONLY the category name.

Text: "{user_input}"
"""

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a strict classifier."},
            {"role": "user", "content": routing_prompt}
        ]
    )

    return response.choices[0].message.content.strip().lower()

In [26]:
print(route_prompt("My Python code has an IndexError"))
print(route_prompt("I was charged twice"))
print(route_prompt("Hello there!"))

technical
billing
general


### Section 4: Test each expert

In [27]:
# Technical Expert Test
response = client.chat.completions.create(
    model=BASE_MODEL,
    temperature=MODEL_CONFIG["technical"]["temperature"],
    messages=[
        {"role": "system", "content": MODEL_CONFIG["technical"]["system_prompt"]},
        {"role": "user", "content": "My Python code gives an IndexError."}
    ]
)

print(response.choices[0].message.content)

**Understanding IndexError in Python**

An `IndexError` in Python occurs when you try to access an element in a sequence (like a list, tuple, or string) using an index that does not exist. This can happen when you're trying to access an element at an index that's out of range.

**Example Code**

Here's a simple example:
```python
my_list = [1, 2, 3]
print(my_list[3])  # IndexError: list index out of range
```
In this example, we're trying to access the element at index 3 in the list `[1, 2, 3]`, but since the list only has 3 elements (at indices 0, 1, and 2), this raises an `IndexError`.

**Solution**

To avoid `IndexError`, you can use one of the following approaches:

1. **Check the length of the sequence**: Before accessing an element, check if the index is within the valid range using the `len()` function.
```python
my_list = [1, 2, 3]
index = 3
if index < len(my_list):
    print(my_list[index])
else:
    print("Index out of range")
```
2. **Use a try-except block**: Wrap the code 

In [28]:
# Billing Expert Test
response = client.chat.completions.create(
    model=BASE_MODEL,
    temperature=MODEL_CONFIG["billing"]["temperature"],
    messages=[
        {"role": "system", "content": MODEL_CONFIG["billing"]["system_prompt"]},
        {"role": "user", "content": "I was charged twice this month."}
    ]
)

print(response.choices[0].message.content)

I'm so sorry to hear that you were charged twice this month. I understand how frustrating and confusing that can be. 

To assist you further, can you please provide me with some more information about the duplicate charges? Could you tell me:

1. What is the date of the first and second charge?
2. What services or products were you charged for on both occasions?
3. Is there a corresponding invoice or billing statement with the duplicate charges?

Your information will help me investigate this issue and resolve it as quickly as possible.


In [29]:
# General Expert Test
response = client.chat.completions.create(
    model=BASE_MODEL,
    temperature=MODEL_CONFIG["general"]["temperature"],
    messages=[
        {"role": "system", "content": MODEL_CONFIG["general"]["system_prompt"]},
        {"role": "user", "content": "What services do you offer?"}
    ]
)

print(response.choices[0].message.content)

I'm happy to help you with a wide range of services. Here are some of the things I can assist you with:

1. **General Knowledge**: I can provide information on various topics such as history, science, technology, health, and more.
2. **Language Translation**: If you need help translating text or want to learn a new language, I can assist you with that.
3. **Text Summarization**: If you have a long piece of text and want a concise summary, I can help you with that.
4. **Conversation**: We can have a conversation on any topic you'd like, and I'll do my best to respond thoughtfully and helpfully.
5. **Writing Assistance**: If you need help with writing a paper, essay, or other document, I can assist you with research, organization, and more.
6. **Calculator and Math**: I can perform basic arithmetic operations, help with algebra, and more.
7. **Trivia and Games**: We can play games like "Would you rather...", "Two truths and a lie", and more.
8. **Jokes and Humor**: If you need a laugh, I

### Section 5: Integrate all the experts

In [30]:
def process_request(user_input):
    category = route_prompt(user_input)

    if category not in MODEL_CONFIG:
        category = "general"

    expert = MODEL_CONFIG[category]

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=expert["temperature"],
        messages=[
            {"role": "system", "content": expert["system_prompt"]},
            {"role": "user", "content": user_input}
        ]
    )

    return category, response.choices[0].message.content

In [41]:
queries = [
    "My login API returns 500 error.",
    "My annual plan was cancelled but I was still billed.",
    "Do you offer student discounts?"
]

for q in queries:
    category, answer = process_request(q)
    print("\nUser Query:", q)
    print("Routed To:", category)
    print("Response:", answer)


User Query: My login API returns 500 error.
Routed To: technical
Response: To troubleshoot a 500 error in your login API, let's follow a step-by-step approach:

1. **Check the logs**: The first step is to check the server logs for any error messages. This will give you a hint about the root cause of the issue. You can usually find these logs in the console or in a file.

2. **Enable debug mode**: Temporarily enable debug mode in your API to get more detailed error messages. This will help you identify the exact line of code where the error is occurring.

3. **Check API request and response**: Verify that the API request is being sent correctly and that the response is being returned correctly. Use a tool like Postman or curl to make a test request to your API.

4. **Check database connection**: If your API is using a database, ensure that the connection is established correctly. If it's not, you'll get a 500 error.

5. **Check for unhandled exceptions**: Make sure that all unhandled e

### Bonus Challenge

In [38]:
def get_bitcoin_price():
    return "The current price of Bitcoin is $65,000."

In [39]:
def process_request(user_input):
    category = route_prompt(user_input)

    if "bitcoin" in user_input.lower():
        return "crypto", get_bitcoin_price()

    if category not in MODEL_CONFIG:
        category = "general"

    expert = MODEL_CONFIG[category]

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=expert["temperature"],
        messages=[
            {"role": "system", "content": expert["system_prompt"]},
            {"role": "user", "content": user_input}
        ]
    )

    return category, response.choices[0].message.content

In [40]:
category, answer = process_request("What is the current price of Bitcoin?")
print("Routed To:", category)
print("Response:", answer)

Routed To: crypto
Response: The current price of Bitcoin is $65,000.
